# 05-supervisor.ipynb
## 멀티 에이전트 패턴 중 하나
- 도메인이 여러개 섞여 있고
- 각 도메인마다 도구가 많고 복잡함
- 하위 담당 에이전트와 사용자가 소통할 필요가 없음
- 단순 도구만 활용할 경우에는 사용 X

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI('gpt-4.1-mini')

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model('gpt-4.1-mini')

In [ ]:
model .invoke('지금의 삼성전자 주가는 뭐야?')

In [ ]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,
    end_time: str,
    attendees: list[str], # ['a@.com', 'b@.com'] 이메일 주소 리스트
    location: str = '', # 위치가 없을 경우, 빈 문자열
):
    '''캘린더 이벤트 생성'''
    return f'이벤트 생성 완료. {title} - {start_time} ~ {end_time}'

@tool
def send_email(
    to: list[str],
    subject: str,
    body: str,
    attendees: list[str]
):
    '''이메일 발송'''
    return f'이메일 발송 완료. {to} - {subject}'

@tool
def get_available_time_slot(
    attendees: list[str],
    date: str,
    duration_minutes: int
):
    '''참가자들이 특정 날짜에 참여 가능한 시간 확인'''
    return ['09:00', '14:00', '16:00']



In [6]:
from langchain.agents import create_agent
from datetime import datetime

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
    f"Now: {datetime.now()}"
)

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slot],
    system_prompt=CALENDAR_AGENT_PROMPT
)


In [10]:
query = '다음주 화요일 2시에 1시간동안 팀 미팅을 잡아줘'

for step in calendar_agent.stream(
    {'messages':[{'role':'user','content':query}]}
):
    for update in step.values():
        for message in update.get('messages',[]):
            message.pretty_print()


================================== Ai Message ==================================
Tool Calls:
  get_available_time_slot (call_Xz4S5P97ZEXjjHkcS0RZnVfI)
 Call ID: call_Xz4S5P97ZEXjjHkcS0RZnVfI
  Args:
    attendees: ['팀']
    date: 2026-03-03
    duration_minutes: 60
================================= Tool Message =================================
Name: get_available_time_slot

["09:00", "14:00", "16:00"]
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_mvnsc5K0iIvlDl6gwScLuMhR)
 Call ID: call_mvnsc5K0iIvlDl6gwScLuMhR
  Args:
    title: 팀 미팅
    start_time: 2026-03-03T14:00:00
    end_time: 2026-03-03T15:00:00
    attendees: ['팀']
    location:
================================= Tool Message =================================
Name: create_calendar_event

이벤트 생성 완료. 팀 미팅 - 2026-03-03T14:00:00 ~ 2026-03-03T15:00:00
================================== Ai Message ==================================

다음주 화요일인 3월 3일 오후 2시부터 3

In [7]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

In [30]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
import os

DB_URI = os.getenv('DB_URI')

db = SQLDatabase.from_uri(DB_URI)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

# Agent 만들기
from langchain.agents import create_agent

# 어떤 DB 사용 하는지 알 수 있는 메서드
dialect = db.dialect


DB_AGENT_SYSTEM_PROMPT = f"""
You are a DB Agent that queries the company employee database.

## Database Schema

Table: teams
- id (SERIAL, PK) -- 팀 고유 ID
- name (VARCHAR) -- 팀명 (Backend, Frontend, Data, DevOps, HR)

Table: employees
- id (SERIAL, PK) -- 직원 고유 ID
- name (VARCHAR) -- 직원 이름
- email (VARCHAR, UNIQUE) -- 이메일 주소
- team_id (INTEGER, FK -> teams.id) -- 소속 팀 ID

## Rules
- You already know the schema. Do NOT query table lists or schema info. Write SQL directly.
- To find team members, JOIN employees with teams: 
  SELECT e.name, e.email FROM employees e JOIN teams t ON e.team_id = t.id WHERE t.name ILIKE '%keyword%'
- Always include email in results.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP).

## Response Format
Return results as JSON:
{{"status": "success", "results": [{{"name": "...", "email": "...", "team": "..."}}], "count": N}}
"""


db_agent = create_agent(
    model, 
    toolkit.get_tools(),
    system_prompt=DB_AGENT_SYSTEM_PROMPT 
    )


In [20]:
query = '백엔드 팀 명단(이름) 조회해줘'

for step in db_agent.stream(
    {'messages':[{'role':'user','content':query}]}
):
    for update in step.values():
        for message in update.get('messages',[]):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_juUzntRepR9v4NCmmen9Y46n)
 Call ID: call_juUzntRepR9v4NCmmen9Y46n
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

courses, customers, dt_demo, employees, lottery_infos, lotto_draws, members, sales, sample, students, students_courses, teams
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_G7Vh3R6QwgGbhEqRBygZratp)
 Call ID: call_G7Vh3R6QwgGbhEqRBygZratp
  Args:
    table_names: employees, teams
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE employees (
	id SERIAL NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(150) NOT NULL, 
	team_id INTEGER NOT NULL, 
	CONSTRAINT employees_pkey PRIMARY KEY (id), 
	CONSTRAINT fk_team FOREIGN KEY(team_id) REFERENCES teams (id) ON

In [9]:
query = '디자인 팀한테 리뷰 리마인더 보내줘'

for step in email_agent.stream(
    {'messages':[{'role':'user','content':query}]}
):
    for update in step.values():
        for message in update.get('messages',[]):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (call_Jc5MyePiVRigzPJD6ePVfw1i)
 Call ID: call_Jc5MyePiVRigzPJD6ePVfw1i
  Args:
    to: ['design@company.com']
    subject: 리뷰 진행 리마인더
    body: 안녕하세요 디자인 팀,

현재 진행 중인 작업에 대한 리뷰를 확인해주시기 바랍니다. 빠른 피드백 부탁드립니다.

감사합니다.
    attendees: ['design@company.com']
================================= Tool Message =================================
Name: send_email

이메일 발송 완료. ['design@company.com'] - 리뷰 진행 리마인더
================================== Ai Message ==================================

디자인 팀에게 리뷰 진행 리마인더 이메일을 보냈습니다. 추가로 보내길 원하시는 내용이 있으면 말씀해주세요.


In [31]:
# 도구 선택 + request에 뭐 넣을지는 슈퍼바이저 에이전트가 결정 

@tool
def manage_email(request:str) :
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """

    result = email_agent.invoke({
        'messages':[{'role':'user','content':request}]
    })
    return result['messages'][-1].text # 마지막 메세지만 관리자 에이전트에게 전달 

@tool
def schedule_event(request:str):
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """

    result = calendar_agent.invoke({
        'messages':[{'role':'user','content':request}]
    })
    return result['messages'][-1].text # 마지막 메세지만 관리자 에이전트에게 전달 


@tool
def db_search_event(request:str):
    """Search the company database for employee and team information.

    Use this when you need to look up:
    - Team members and their email addresses
    - Individual employee info (name, email, team)

    This tool MUST be called before sending emails or scheduling events for a team,
    so you can get the correct recipient email addresses.

    Available teams: Backend, Frontend, Data, DevOps, HR

    Input: Natural language query (e.g., 'Backend팀 구성원 조회', 'find all members of the HR team')
    Returns: JSON with employee names, emails, and team info.
    """
    result = db_agent.invoke({
        'messages':[{'role':'user','content':request}]
    })
    return result['messages'][-1].text # 마지막 메세지만 관리자 에이전트에게 전달 


In [37]:
SUPERVISOR_PROMPT = f'''You are a smart personal assistant that coordinates tasks using three tools:
- db_search_event: Search employee/team info from the company database
- schedule_event: Create calendar events
- manage_email: Send emails

## Workflow Rules
1. When a request involves a team or department, ALWAYS call db_search_event FIRST to get member names and email addresses.
2. Use the retrieved email addresses for both calendar invites (attendees) and email recipients.
3. Think step-by-step about the correct order of tool calls before acting.

## Available Teams
Backend, Frontend, Data, DevOps, HR

## Example Flow
User: "다음주 화요일 오후 2시에 Backend팀 미팅 잡고 메일 보내줘"
→ Step 1: db_search_event("Backend팀 구성원 조회")
→ Step 2: schedule_event("다음주 화요일 14:00~15:00 Backend팀 미팅, 참석자: [조회된 이메일들]")
→ Step 3: manage_email("Backend팀 구성원들에게 미팅 안내 메일 발송, 수신자: [조회된 이메일들]")

Always respond in the user's language.
Now: {datetime.now()}
'''

supervisor_agent = create_agent(
    model,
    tools =[schedule_event, manage_email, db_search_event],
    system_prompt=SUPERVISOR_PROMPT
)

In [13]:
query = '내일 오전 9시에 팀 아침회의 잡아줘'

for step in supervisor_agent.stream(
    {'messages':[{'role':'user','content':query}]}
):
    for update in step.values():
        for message in update.get('messages',[]):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_d3isUIoMZ6N9jMHRTkAPOtUP)
 Call ID: call_d3isUIoMZ6N9jMHRTkAPOtUP
  Args:
    request: 팀 아침회의 내일 오전 9시
================================= Tool Message =================================
Name: schedule_event

팀 아침회의를 내일 오전 9시부터 9시 30분까지 예약했습니다. 추가로 참석자나 장소를 지정하고 싶으신가요?
================================== Ai Message ==================================

내일 오전 9시부터 9시 30분까지 팀 아침회의를 예약했습니다. 참석자나 장소를 지정하고 싶으신가요?


In [ ]:
# query = '다음주 화요일 오후 2시부터 1시간 30분동안 HR팀과 미팅 잡고 메일 보내놔'

for step in supervisor_agent.stream(
    {'messages':[{'role':'user','content':query}]}
):
    for update in step.values():
        for message in update.get('messages',[]):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  db_search_event (call_0YpcITtyGQcuk7bBLCvJLKpA)
 Call ID: call_0YpcITtyGQcuk7bBLCvJLKpA
  Args:
    request: HR팀 구성원 조회
================================= Tool Message =================================
Name: db_search_event

{"status": "success", "results": [{"name": "정인사1", "email": "hr1@company.com", "team": "HR"}, {"name": "정인사2", "email": "hr2@company.com", "team": "HR"}, {"name": "정인사3", "email": "hr3@company.com", "team": "HR"}, {"name": "정인사4", "email": "hr4@company.com", "team": "HR"}, {"name": "정인사5", "email": "hr5@company.com", "team": "HR"}], "count": 5}
================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_yDNIuEzOaL5h1tkRqOTxTl4w)
 Call ID: call_yDNIuEzOaL5h1tkRqOTxTl4w
  Args:
    request: 다음주 화요일 오후 2시부터 3시 30분까지 HR팀 미팅, 참석자: hr1@company.com, hr2@company.com, hr3@company.com, hr4@company.com, hr5@company.com
  manage_